In [1]:
import os 
import tarfile
import urllib.request
import tensorflow as tf
import numpy as np
import re
import string
from random import randint
url="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
filepath='aclImdb_v1.tar.gz'
if not os.path.isfile(filepath):
    print("download--")
    result=urllib.request.urlretrieve(url,filepath)
    print(result)
else:
    print("already!")
if not os.path.exists("aclImdb"):
    tfile=tarfile.open(filepath,"r:gz")
    print("extracting...")
    result=tfile.extractall()
else:
    print("already well")

D:\MyDownloads\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\MyDownloads\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\MyDownloads\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\MyDownloads\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:461: FutureWarning: Passing (

already!
already well


In [2]:
def remove_tags(text):
        re_tag=re.compile(r'<[^>]+>')
        return re_tag.sub('',text)

In [3]:
def read_files(filetype):
        path="/aclImdb/"
        file_list=[]
        positive_path=path+filetype+"/pos/"
        for f in os.listdir(positive_path):
            file_list+=[positive_path+f]
        pos_files_num=len(file_list)
        
        negative_path=path+filetype+"/neg/"
        for f in os.listdir(negative_path):
            file_list+=[negative_path+f]
        neg_files_num=len(file_list)-pos_files_num
        #print(file_list)
        all_labels=([[1,0]]*pos_files_num+[[0,1]]*neg_files_num)
        all_text=[]
        for fi in file_list:
            with open(fi,encoding='utf8') as file_input:
                all_text+=[remove_tags(" ".join(file_input.readlines()))]
        return all_labels,all_text

In [4]:
train_texts,train_labels=read_files("train")
test_texts,test_labels=read_files("test")

In [5]:
#建立词汇词典tokSen
#! pip install keras==2.0.6
import keras
from keras.preprocessing.text import Tokenizer
token=Tokenizer(num_words=4000)
token.fit_on_texts(train_labels)
y_train=train_texts
y_test=test_texts
train_sequence=token.texts_to_sequences(train_labels)
test_sequence=token.texts_to_sequences(test_labels)
x_train=keras.preprocessing.sequence.pad_sequences(train_sequence,
                                                  padding='post',
                                                  truncating='post',
                                                  maxlen=400)
x_test=keras.preprocessing.sequence.pad_sequences(test_sequence,
                                                  padding='post',
                                                  truncating='post',
                                                  maxlen=400)

Using TensorFlow backend.


In [7]:
model=keras.models.Sequential()
model.add(keras.layers.Embedding(output_dim=32,
                                input_dim=4000,
                                input_length=400))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=8)))
model.add(keras.layers.Dense(units=32,activation='relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(units=2,activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
hist=model.fit(x_train,y_train,validation_split=0.2,epochs=5,batch_size=128,verbose=1)

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 73s - loss: 0.6067 - acc: 0.6734 - val_loss: 0.6280 - val_acc: 0.7510
Epoch 2/5
20000/20000 [==============================] - 80s - loss: 0.4068 - acc: 0.8366 - val_loss: 0.8752 - val_acc: 0.6364
Epoch 3/5
20000/20000 [==============================] - 79s - loss: 0.3467 - acc: 0.8672 - val_loss: 0.6768 - val_acc: 0.7120
Epoch 4/5
20000/20000 [==============================] - 80s - loss: 0.3237 - acc: 0.8770 - val_loss: 0.4478 - val_acc: 0.8248
Epoch 5/5
20000/20000 [==============================] - 79s - loss: 0.2829 - acc: 0.8985 - val_loss: 0.5222 - val_acc: 0.7922
